<a href="https://colab.research.google.com/github/mistryishan25/Comprehensive-Project-SEM8/blob/master/models/1_0_ish_Node_Classifier_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resource dump:
1. Which Free GPUs - [Link](https://towardsdatascience.com/free-gpus-for-training-your-deep-learning-models-c1ce47863350)

In [ ]:
!pip install wandb

In [ ]:
!pip install convokit

In [ ]:
!pip install contractions

In [ ]:
!pip install transformers

### Cite this in the final paper: 

Characterizing Online Discussion Using Coarse Discourse Sequences,
Amy Zhang,Bryan Culbertson,Praveen Paritosh


Bibtex
@inproceedings{46055,
title	= {Characterizing Online Discussion Using Coarse Discourse Sequences},
author	= {Amy Zhang and Bryan Culbertson and Praveen Paritosh},
year	= {2017}
}

Reference : [Link](https://colab.research.google.com/github/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/Introduction_to_ConvoKit.ipynb#scrollTo=kHB78-JtViKt)

In [ ]:
BASE_CORPUS = Corpus(download("reddit-coarse-discourse-corpus"));
#corpus.print_summary_stats();

### What each utterance contains?
1. comment_depth: depth of the comment, 0 if the utterance is the top-level post itself.
2. majority type: discourse action type by one of the following: question, answer, announcement, agreement, appreciation, disagreement, elaboration, humor
3. annotation_types (list of annotation types by three annotators)
4. majority_link : link in relation to previous post, none if no relation with previous comment
5. annotation_links (list of annotation links by three annotators)


In [ ]:
utt_5 = []
for i in range(5):
    utt = BASE_CORPUS.random_utterance()
    utt_5.append(utt)
    print(utt.text)
    print("-"*40)
    # print(utt.meta)

y'all realize that fasttech is in hong kong? they probably pick up overstock from factories in shenzhen.
----------------------------------------
>You can see in dim light within 60 feet of you as if it were bright light, and in darkness as if it were dim light. You can’t discern color in darkness, only shades of gray.

>In a lightly obscured area, such as dim light, patchy fog, or moderate foliage, creatures have disadvantage on Wisdom (Perception) checks that rely on sight.

>A heavily obscured area—such as darkness, opaque fog, or dense foliage—blocks vision entirely. A creature in a heavily obscured area effectively suffers from the blinded condition (see appendix A) when trying to see something in that area.

So in darkness with darkvision you treat it as dim light, it only modifies Wisdom(perception) checks by giving them disadvantage. Remember disadvantage on passive checks is a -5 penalty, so passive perception is 5 lower than normal.

No other penalties to combat rolls.

Edit:

### Explore the conversations

1. [ ] How many emojis do we actually have? Are they used instead of words?
2. [ ] Explore the distribution of the labels. 

- We would be needing only the text and the meta.
- Vector attribute stays we might add the vector data for each utterance for other cases- [Link](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/vectors/vector_demo.ipynb)

In [ ]:
df = BASE_CORPUS.get_utterances_dataframe()
df.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.post_depth,meta.majority_type,meta.majority_link,meta.annotation-types,meta.annotation-links,meta.ups,vectors
id,,,,,,,,,,,,
t3_1bx6qw,None,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,DTX120,None,t3_1bx6qw,0,announcement,none,"[announcement, announcement, announcement]","[none, none, none]",3,[]
t1_c9b2nyd,None,I've wanted to watch this for a long time. I w...,mcgrewf10,t3_1bx6qw,t3_1bx6qw,1,elaboration,t3_1bx6qw,"[agreement, elaboration, elaboration]","[t3_1bx6qw, t3_1bx6qw, t3_1bx6qw]",2,[]
t1_c9b30i1,None,You strike me as the type who would appreciate...,DTX120,t1_c9b2nyd,t3_1bx6qw,2,elaboration,t1_c9b2nyd,"[elaboration, elaboration, elaboration]","[t1_c9b2nyd, t1_c9b2nyd, t1_c9b2nyd]",1,[]
t1_c9b6sj0,None,"Yeah, I've always heard that Altman was famous...",mcgrewf10,t1_c9b30i1,t3_1bx6qw,3,elaboration,t1_c9b30i1,"[agreement, elaboration, elaboration]","[t1_c9b30i1, t1_c9b30i1, t1_c9b30i1]",1,[]
t3_omv7p,None,"Alright guys, little background about myself. ...",Keatonus,None,t3_omv7p,0,announcement,none,"[announcement, announcement, announcement]","[none, none, none]",6,[]


In [ ]:
# Yes indeed these are the number of utterances in the entire dataset.
len(df)

115827

In [ ]:
# Meh not important here anyways
df = df.drop(["timestamp"], axis=1)

# Vectors are also null in a way as they will updated later on with the choice of embedding in the metadata

In [ ]:
# Checking for missing values : 

def missing_values(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    # Seperating them into a new df for use later on while testing
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})
    return missing_value_df

### List of Cleaning to be done
1. [x] Clean URls - Convention is [text]\(URL)
2. [x] Do not seperate the quoted text - "> blah /n/n
3. [x] Removing special characters 
4. [ ] Speling correction? What about the ones that convey info?
5. [ ] Deal with Emojis and emoticons - [emot lib](https://medium.com/geekculture/text-preprocessing-how-to-handle-emoji-emoticon-641bbfa6e9e7) 
6. [x] Contractions need to be taken care of -
7. [x] Remove /n

Using Regex to do simple cleaning based on symbols - [Documentation](https://docs.python.org/3/library/re.html#re.sub) and many stack-overflow pieces and articles

In [ ]:
def clean(text, newline=True, quote=True, bullet_point=True,dates=True,
          link=True, strikethrough=True, spoiler=True, heading=True, emoji=True, emoticon=True, contraction=True):
    
    # Newlines we dont need - only 
    if newline:
        text = re.sub(r'\n+', ' ', text)
        # Remove the many " " that we replaced in the last steo
        text = text.strip()
        text = re.sub(r'\s\s+', ' ', text)

    # > are for the qouted texts from the main comment or the reply
    if quote:
        text = re.sub(r'>', '', text)

    # Bullet points/asterisk are used for markdown like - bold/italic - Could create trouble in parsing? idk
    if bullet_point:
        text = re.sub(r'\*', '', text)
        text = re.sub('&amp;#x200B;', '', text)

    # []() Link format then we remove both the tag/placeholder and the link
    if link:
        text = re.sub(r"http\S+", '', text)
        text = re.sub(r'\[.*?\]\(.*?\)', '', text)

    # Strikethrough
    if strikethrough:
        text = re.sub('~', '', text)

    # Spoiler, which is used with < less-than (Preserves the text)
    if spoiler:
        text = re.sub('&lt;', '', text)
        text = re.sub(r'!(.*?)!', r'\1', text)

    # Heading to be removed as there are these markdown style features in reddit too
    if heading:
        text = re.sub('#', '', text)
        
    if emoji:
    # Implement the emoji scheme here. 
    # Makes more sense for the node feature but might as well import that function here if ready    
        pass
    if dates:
        text = re.sub(r'(\d+/\d+/\d+)', '', text)
    if emoticon:
    # Implement the emoticon scheme here. 
    # Makes more sense for the node feature but might as well import that function here if ready 
        pass
    
    #Needs to be the last step in the process
    if contractions:
        text = contractions.fix(text)
    #print("Running")    
    return text

### Ways of transformation to clean 
1.  Clean() then TextCleaner -----Meh
2. TextCleaner then Clean() ---------Meh 
3. Clean -------------------------MeH
4. TextCleaner ------------------Meh
5. Clean : Lambda ---------------Bingo

The convo-kit has good interoperability with the sklearn package and hence we would have to exploit the sklearn stuff to do the transformation in a single pass instead of iterating over all the utterances - [CovoKit](https://convokit.cornell.edu/documentation/architecture.html#transformer)

In [ ]:
# Option 1 - Clean -> TextCleaner
corpus_1 = textCleaner.TextCleaner(text_cleaner= clean, input_field = "text").transform(BASE_CORPUS);
corpus_2 = textCleaner.TextCleaner(input_field= "text").transform(corpus_1);
df_1 = corpus_2.get_utterances_dataframe()

In [ ]:
missing_1 = missing_values(df_1)
missing_1

In [ ]:
# Option 2 TextCleaner -> Clean
corpus_3 = textCleaner.TextCleaner(input_field= "text").transform(BASE_CORPUS)
corpus_4 = textCleaner.TextCleaner(text_cleaner= clean, input_field = "text").transform(corpus_3)
df_2 = corpus_4.get_utterances_dataframe()

In [ ]:
missing_2 = missing_values(df_2)
missing_2

In [ ]:
# Option 3 Clean
corpus_5 = textCleaner.TextCleaner(text_cleaner= clean, input_field = "text").transform(BASE_CORPUS)
df_3 = corpus_5.get_utterances_dataframe()

In [ ]:
missing_3 = missing_values(df_3)
missing_3

In [ ]:
# Option 4 TextCleaner
corpus_6 = textCleaner.TextCleaner(input_field= "text").transform(BASE_CORPUS)
df_4 = corpus_6.get_utterances_dataframe()


100/115827 utterances processed
200/115827 utterances processed
300/115827 utterances processed
400/115827 utterances processed
500/115827 utterances processed
600/115827 utterances processed
700/115827 utterances processed
800/115827 utterances processed
900/115827 utterances processed
1000/115827 utterances processed
1100/115827 utterances processed
1200/115827 utterances processed
1300/115827 utterances processed
1400/115827 utterances processed
1500/115827 utterances processed
1600/115827 utterances processed
1700/115827 utterances processed
1800/115827 utterances processed
1900/115827 utterances processed
2000/115827 utterances processed
2100/115827 utterances processed
2200/115827 utterances processed
2300/115827 utterances processed
2400/115827 utterances processed
2500/115827 utterances processed
2600/115827 utterances processed
2700/115827 utterances processed
2800/115827 utterances processed
2900/115827 utterances processed
3000/115827 utterances processed
3100/115827 utteran

In [ ]:
missing_4 = missing_values(df_4)
missing_4

,column_name,percent_missing
timestamp,timestamp,100.000000
text,text,0.000000
speaker,speaker,0.000000
reply_to,reply_to,8.187210
conversation_id,conversation_id,0.000000
meta.post_depth,meta.post_depth,0.000000
meta.majority_type,meta.majority_type,10.599428
meta.majority_link,meta.majority_link,1.799235
meta.annotation-types,meta.annotation-types,0.000000
meta.annotation-links,meta.annotation-links,0.000000


In [ ]:
# Lucky to find that the first utterance is a problem 
# BASE_CORPUS Version
[df.head(1)][0]["text"][0]

In [ ]:
[df_1.head(1)][0]["text"][0]

In [ ]:
[df_2.head(1)][0]["text"][0]

In [ ]:
clean([df_2.head(1)][0]["text"][0])

# Returns None
[df_3.head(1)][0]["text"][0] 

In [ ]:
assert df_4["text"].all() == df_2["text"].all() == df_1["text"].all() 
# Damn why God Why did I waste time then? 

In [ ]:
# Option 5 - Use lambda Function
df["text_clean"] = df["text"].apply(lambda row : clean(row))

In [ ]:
assert df_4["text"][0] == df["text_clean"][0] 
# Yes it should not be truee so wohhoo!!!

AssertionError: 

In [ ]:
df["text"][0]

"4/7/13  \n\n7/27/12  \n\nhttp://www.imdb.com/title/tt0073440/reference\n\nIt was only a few minutes into Robert Altman's homespun epic *Nashville* that I got the feeling I was watching a great movie. By the end it could not be denied. Now I'm sure it helps that I'm a musician, since this created an immediate connection to the subject matter. I spent a portion of the movie with my Telecaster in my lap trying to play along with the characters who all seem to be really playing and singing these songs. However I also am *not* a fan of country western, so that could have easily been a turn off.  \n\nTo begin describing the action in the film is daunting. I can't even process a lot of what I saw. This movie is extremely dense, and the first 30 minutes or so are spent just trying to figure out who people are. Their relationships to one another - some of which are purely incidental - slowly become clear as things progress. It's an ensemble cast with no clear lead and lots of overlapping conve

In [ ]:
df["text_clean"][0]

"   It was only a few minutes into Robert Altman's homespun epic Nashville that I got the feeling I was watching a great movie. By the end it could not be denied. Now I am sure it helps that I am a musician, since this created an immediate connection to the subject matter. I spent a portion of the movie with my Telecaster in my lap trying to play along with the characters who all seem to be really playing and singing these songs. However I also am not a fan of country western, so that could have easily been a turn off. To begin describing the action in the film is daunting. I cannot even process a lot of what I saw. This movie is extremely dense, and the first 30 minutes or so are spent just trying to figure out who people are. Their relationships to one another - some of which are purely incidental - slowly become clear as things progress. It is an ensemble cast with no clear lead and lots of overlapping conversations. Some developments all tie together in the end. Some seem to be kin

### Prep for the Bert thingy

In [ ]:
# making a deep copy coz I think pandas works with shallow copies by default! 
train_df = df[["text_clean", "meta.majority_type", "meta.annotation-types"]].copy() 
train_df.rename(columns={"meta.majority_type" : "label", "meta.annotation-types" : "options"}, inplace=True)

In [ ]:
# no need of index
train_df.reset_index(drop=True, inplace=True)

In [ ]:
train_df.head()

,text_clean,label,options
0,It was only a few minutes into Robert Altma...,announcement,"[announcement, announcement, announcement]"
1,I have wanted to watch this for a long time. I...,elaboration,"[agreement, elaboration, elaboration]"
2,You strike me as the type who would appreciate...,elaboration,"[elaboration, elaboration, elaboration]"
3,"Yeah, I have always heard that Altman was famo...",elaboration,"[agreement, elaboration, elaboration]"
4,"Alright guys, little background about myself. ...",announcement,"[announcement, announcement, announcement]"


In [ ]:
# Dealing with categorical data
codes, labels = pd.factorize(train_df["label"])

In [ ]:
train_df["label"] = pd.Series(data=codes).copy()

In [ ]:
train_df.head()

,text_clean,label,options
0,It was only a few minutes into Robert Altma...,0,"[announcement, announcement, announcement]"
1,I have wanted to watch this for a long time. I...,1,"[agreement, elaboration, elaboration]"
2,You strike me as the type who would appreciate...,1,"[elaboration, elaboration, elaboration]"
3,"Yeah, I have always heard that Altman was famo...",1,"[agreement, elaboration, elaboration]"
4,"Alright guys, little background about myself. ...",0,"[announcement, announcement, announcement]"


In [ ]:
# Checking if the number of tokens exceeds 512 and by what amount?
import re
import pandas as pd

In [ ]:
train_df["label"].value_counts()
#-1 means that these are the labels that are missing

 5    41162
 1    19258
 4    17594
-1    12277
 3     8710
 6     5040
 8     3422
 2     2417
 9     2049
 0     2002
 7     1896
Name: label, dtype: int64

In [ ]:
train_df["num_tokens"] = train_df["text_clean"].apply(lambda sent : len(re.findall(r'\w+', sent)))

In [ ]:
# Removed all the token that exceeded the limit
train_df = train_df[train_df["num_tokens"]<510]

In [ ]:
len(train_df["num_tokens"])

115374

In [ ]:
# Checks if things are in order so far
assert len(train_df_labeled.loc[train_df_labeled["label"]<0]) == 0 
assert len(train_df)-len(train_df_missing) == len(train_df_labeled)

In [ ]:
train_df_labeled.head()

,text_clean,label,options,num_tokens
0,It was only a few minutes into Robert Altma...,0,"[announcement, announcement, announcement]",447
1,I have wanted to watch this for a long time. I...,1,"[agreement, elaboration, elaboration]",20
2,You strike me as the type who would appreciate...,1,"[elaboration, elaboration, elaboration]",74
3,"Yeah, I have always heard that Altman was famo...",1,"[agreement, elaboration, elaboration]",21
4,"Alright guys, little background about myself. ...",0,"[announcement, announcement, announcement]",102


In [ ]:
train_df_labeled["label"].value_counts()

5    41162
1    19258
4    17594
3     8710
6     5040
8     3422
2     2417
9     2049
0     2002
7     1896
Name: label, dtype: int64

In [ ]:
train_df_labeled["num_tokens"].value_counts()

1      3110
0      2863
6      2663
5      2578
7      2486
       ... 
478       1
506       1
473       1
482       1
504       1
Name: num_tokens, Length: 502, dtype: int64

In [ ]:
test = train_df_labeled.iloc[3333]
print(test["text_clean"])
print(test["num_tokens"])
print(test["label"])
print(labels)

Stress
1
5
Index(['announcement', 'elaboration', 'humor', 'appreciation', 'question',
       'answer', 'agreement', 'negativereaction', 'disagreement', 'other'],
      dtype='object')


In [ ]:
train_df_missing["label"].value_counts()

-1    12244
Name: label, dtype: int64

### BERT variants
All the BERTs that have been trained on conversational data can be more benifical that just the vanilla bert trained on something like Wiki corpus.

1. [ ] MPC-BERT : Trained on Multi-Party comm
2. [ ] CS-BERT : Customer Service
3. [ ] 

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#Original:  Our friends won't buy this analysis, let alone the next one we propose.
#Tokenized:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
#Token IDs:  [2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]


Loading BERT tokenizer...


In [ ]:
max_len = 0

# For every sentence...
for sent in all_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  1996


For the Hyper- for the sweep configurations we should follow : [Link](https://datascience.stackexchange.com/questions/64583/what-are-the-good-parameter-ranges-for-bert-hyperparameters-while-finetuning-it)

In [ ]:
import wandb
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [ 5e-5, 3e-5, 2e-5]
        },
        'batch_size': {
            'values': [16, 32]
        },
        'epochs':{
            'values':[2, 3, 4]
        }
    }
}
sweep_defaults = {
    'learning_rate': 5e-5,
    'batch_size': 32,
    'epochs':2
}

sweep_id = wandb.sweep(sweep_config)


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: jnxpyr46
Sweep URL: https://wandb.ai/mistr/uncategorized/sweeps/jnxpyr46


#### Tokenization
To follow a general convention that the sizes should be in powers of 2, we’ll choose the closest number that is a power of 2, i.e, 64.

Now, we’re ready to perform the real tokenization. But as we’re using transformers, we can use an inbuilt function tokenizer.encode_plus which automates all of the following tasks:

1. Split the sentence into tokens.
2. Add the special `[CLS]` and `[SEP]` tokens.
3. Map the tokens to their IDs.
4. Pad or truncate all sentences to the same length.
5. Create the attention masks which explicitly differentiate real tokens from `[PAD]` tokens.


In [ ]:
#Remember we are only working iwth the training set first as the label comparision for the missing labels 10% is still left - so for now this(train_df_labeled) would be ou entire dataset 
labeled_sentences = train_df_labeled["text_clean"]
given_labels = train_df_labeled["label"]

In [ ]:
labeled_sentences.iloc[35]

'Whew! Cannot afford that one. I am lookin for the base model that sells on impactguns.com for like $540.'

In [ ]:
assert len(labeled_sentences) == len(given_labels)

In [ ]:
import torch
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in labeled_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(given_labels, dtype=torch.int)

In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', labeled_sentences[0])
print('Token IDs:', input_ids[0])

Original:     It was only a few minutes into Robert Altman's homespun epic Nashville that I got the feeling I was watching a great movie. By the end it could not be denied. Now I am sure it helps that I am a musician, since this created an immediate connection to the subject matter. I spent a portion of the movie with my Telecaster in my lap trying to play along with the characters who all seem to be really playing and singing these songs. However I also am not a fan of country western, so that could have easily been a turn off. To begin describing the action in the film is daunting. I cannot even process a lot of what I saw. This movie is extremely dense, and the first 30 minutes or so are spent just trying to figure out who people are. Their relationships to one another - some of which are purely incidental - slowly become clear as things progress. It is an ensemble cast with no clear lead and lots of overlapping conversations. Some developments all tie together in the end. Some seem

In [ ]:
import torch

In [ ]:
given_labels.reset_index(drop=True, inplace=True)
given_labels

0         0
1         1
2         1
3         1
4         0
         ..
103125    4
103126    5
103127    5
103128    5
103129    1
Name: label, Length: 103130, dtype: int64

In [ ]:
labels = torch.tensor(given_labels, dtype=torch.int)

In [ ]:
labels

NameError: ignored